|   
|:--| 
|<img src ='https://bit.ly/39XPeCc' width=200 align='left'>
|<h1>    starcraft_LGBM_0415_startover_r01.ipynb
</h1>    

### **데이터 분리** (parametric study)
> - 80% 20%  Train/Test 자체 Test Data set 으로 모델을 검증 한다.
> - Sklearn.train_data_split() 을 이용해서 분리한다.
> - 파라메터를 바꿔 가면서 모델의 예측율을 사전 검증

## gDrive 연결
> - drive 폴더가 붙은 것을 확인한다.
> - drwx------ 4 root root 4.0K Mar 22 11:21 drive

In [1]:
from google.colab import drive 
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## 모듈 Install / Import 

In [2]:
# 코랩에 없는 라이브러리 설치해주기
!pip install bayesian-optimization

In [0]:
""" 
# WAY-01 = LGBM, WAY-02 = XGBM
"""

init_points, n_iter = (20, 50)                      
filename_submit = f"submission_0415_LGBM_{init_points}_{n_iter}_startover_r01.csv"
file_save = False

## 사용함수의 정의

In [0]:
""" 
# 기본적인 module import 위치로 작업폴더 변경 getcwd() --> chidr()
# HOME 을 지정하고 작업폴더를 HOME 으로 변경.

# # for PC
# import os
# HOME= 'dacon_competition_2020'
# dir_base = "".join(os.getcwd().partition(HOME)[:2]) + "\\"
# os.chdir(dir_base)  
"""   

# for gDrive
import os
HOME = '/content/drive/My Drive/Colab Notebooks/'
os.chdir(HOME)

In [5]:
from _assets.config_dirs import *
from _assets.modules import *
from _assets.module_lgbm_model import *


# 화일분석에 필요한 공동폴더를 등록합니다.
# - echo = True : SYS.PATH INSERT 상황 보여줌


# OS 화일 및 DF 정보조회를 위한 탐색 모듈


# LGBM_CV 모델 - Dacon Baseline 참조



In [0]:
import os
import sys           
import random 
import warnings          

import numpy as np                          # 계산 라이브러리
import pandas as pd                         # 데이터 분석 라이브러리
import matplotlib.pyplot as plt             # * 그래프 이미지
import lightgbm as lgb                      # LightGBM 라이브러리 

from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정

warnings.filterwarnings("ignore")           # 경고 문구 미표시

## 데이터를 불러온다

In [7]:
%%time
"""
# show_infoDF_from(x_train) # *** DATA SHAPE = [ 38,872 x 1,579 ] 
# to <class 'pandas.core.frame.DataFrame'>
# show_infoDF_from(y_train) # *** DATA SHAPE = [ 38,872 x 3 ]
# to <class 'numpy.ndarray'> ...  array([1, 1, 0, ..., 0, 1, 0])
"""
# x_train = pd.read_csv(dir_base + raw + 'xtra_remake_xtrain_final.csv')  
# ytrain = pd.read_csv(dir_base + remake + 'df_ytrain_remake.csv')  
# y_train = ytrain['winner'].values    


_df1 = pd.read_csv(dir_base + raw + 'X1_train.csv')

x_train = _df1.iloc[:,:-1]
y_train = _df1.loc[:, 'Y'].to_numpy()     

# x_train.shape, y_train.shape  # ((31097, 27), (31097,))
# type(y_train)                 # numpy.ndarray
# y_train                       # array([1, 1, 0, ..., 1, 1, 0])

CPU times: user 77.9 ms, sys: 51.1 ms, total: 129 ms
Wall time: 998 ms


## LGBM Classifier 모델 적용
 -  Light Gradient Boost Method 적용 - [출처] :  YW & YY's Python, Machine & Deep Learning
 > - Light GBM: A Highly Efficient Gradient Boosting Decision Tree 논문 리뷰
 > - [Light GBM 모델 설명 및 사용법](https://greeksharifa.github.io/machine_learning/2019/12/09/Light-GBM/)

 > 다중 분류, 클릭 예측, 순위 학습 등에 주로 사용되는 Gradient Boosting Decision Tree (GBDT)는 굉장히 유용한 머신러닝 알고리즘이며, XGBoost나 pGBRT 등 효율적인 기법의 설계를 가능하게 하였다. 이러한 구현은 많은 엔지니어링 최적화를 이룩하였지만 고차원이고 큰 데이터 셋에서는 만족스러운 결과를 내지 못하는 경우도 있었다. 왜냐하면 모든 가능한 분할점에 대해 정보 획득을 평가하기 위해 데이터 개체 전부를 스캔해야 했기 때문이다. 이는 당연하게도, 굉장히 시간 소모적이다.


In [8]:
%%time
"""
# 처음 5회 랜덤 값으로 score 계산 후, 30회 최적화 = Original Baseline!
# lgbBO.maximize(init_points=5, n_iter=30) 
"""

var_fixed = partial(                        # 모델과 관련없는 변수 고정
                  lgb_cv, 
                  x_data=x_train, 
                  y_data=y_train, 
                  n_splits=5, 
                  output='score',
                ) 

lgbBO = BayesianOptimization(                # 베이지안 최적화 범위 설정
      var_fixed, 
      {
          'colsample_bytree': (0, 1),        # colsample_bytree, 범위(0~1)
          'learning_rate': (0.0001, 0.1),    # learning_rate,    범위(0.0001~0.1)
          'n_estimators': (16, 62),        # n_estimators,     범위(16~1024)
          'num_leaves': (16, 62),          # num_leaves,       범위(16~1024)
          'reg_alpha': (0, 10),              # reg_alpha,        범위(0~10)
          'reg_lambda': (0, 50),             # reg_lambda,       범위(0~50)
          'subsample': (0, 1),               # subsample,        범위(0~1)
      }, 
        # random_state=random.randrange(50000)   # 초기 값  = 50000
        random_state=random.randrange( 4321)   # 시드 고정 = 4321
  )

lgbBO.maximize(init_points, n_iter)

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.6412   |  0.2254   |  0.09283  |  43.58    |  36.94    |  3.864    |  3.129    |  0.974    |
|  2        |  0.6379   |  0.7366   |  0.07346  |  23.55    |  49.91    |  7.385    |  26.83    |  0.6006   |
|  3        |  0.6309   |  0.153    |  0.07564  |  29.61    |  20.33    |  1.83     |  41.5     |  0.5567   |
|  4        |  0.6423   |  0.4161   |  0.07741  |  59.93    |  46.74    |  9.412    |  48.66    |  0.6822   |
|  5        |  0.6337   |  0.2152   |  0.06103  |  34.0     |  55.15    |  1.045    |  0.4995   |  0.8293   |
|  6        |  0.6339   |  0.6926   |  0.02188  |  30.43    |  19.24    |  7.054    |  12.66    |  0.8937   |
|  7        |  0.6357   |  0.3423   |  0.06896  |  30.85    |  50.83    |  4.221    |  42.73    |  0.6863   |
|  8      

## 모델 학습 및 검증
- Model Tuning & Evaluation

> 1. AUC가 가장 높은 하이퍼 파라미터를 사용해 최종 모델을 얻습니다.
> 1. 훈련 세트와 같은 방법으로 테스트 세트에서 Feature를 추출합니다.
> 1. 최종 모델을 사용해 예측을 수행합니다.
> 1. 예측 결과를 submission.csv로 저장합니다.

In [0]:
"""
# Light Gradient Boost Method 적용
# Light GBM: A Highly Efficient Gradient Boosting Decision Tree 논문 리뷰
# https://greeksharifa.github.io/machine_learning/2019/12/09/Light-GBM/)
"""
params = lgbBO.max['params']

models = lgb_cv(
        num_leaves=       params['num_leaves'], 
        learning_rate=    params['learning_rate'], 
        n_estimators=     params['n_estimators'], 
        subsample=        params['subsample'], 
        colsample_bytree= params['colsample_bytree'], 
        reg_alpha=        params['reg_alpha'], 
        reg_lambda=       params['reg_lambda'], 
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='model',
    )

## TEST를 풀기위해, 학습데이터로 변경 (data_preps)
- df_xtrain_final.shape   # (38872, 1579)
- df_xtest_final.shape   # (16787, 1579)

In [0]:
"""
# 테스트용 데이터를 전처리 해서 분석 준비 ... (시간소요) 미리 저장한 화일을 불러온다
# 저장했으면 비활성 처리
# x_test, _ = data_preparation(test, answer=False)                 # [ 28,714,849 x 6 ] ... 30%
# x_test.to_csv(dir_base + remake + 'df_xtest_remake.csv')
# x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')  # [  16,787 x 27 ]
"""
# x_test = pd.read_csv(dir_base + remake + 'xtra_remake_xtest_final.csv')    # *** DATA SHAPE = [ 16,787 x 27 ]

_df2 = pd.read_csv(dir_base + raw + 'X1_test.csv')
x_test = _df2.iloc[:,:-1]
y_test = _df2.loc[:, 'Y']

In [11]:
x_test.shape, y_test.shape  # ((31097, 27), (31097,))
type(y_test)                 # numpy.ndarray
y_test                       # array([1, 1, 0, ..., 1, 1, 0])

0       0
1       1
2       0
3       0
4       0
       ..
7770    1
7771    0
7772    0
7773    1
7774    0
Name: Y, Length: 7775, dtype: int64

## 예측결과 Submission 화일 만들기

In [0]:
"""
# pd.read_csv('data/sample_submission.csv', index_col=0)
# sample_submission = pd.read_csv(dir_base + sub_base + "sample_submission.csv")
"""

preds = []

for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis=0)

In [0]:
"""
# submission = pd.read_csv(dir_base + raw + "sample_submission.csv", index_col='game_id') 
# submission = pd.DataFrame(pred, columns=['winner'])

# submission['winner'] = pred
"""


submission = pd.DataFrame(pred, columns=['winner'])
# submission.to_csv(dir_base + submit + filename_submit, index=False)

In [17]:
"""
# submission = pd.read_csv(dir_base + raw + "sample_submission.csv") 
# submission['winner'] = pred
"""
if file_save:
    submission.to_csv(dir_base + submit + filename_submit, index=False)
else:
    print('*** SAVING FILE SKIPPED!! ***')   

*** SAVING FILE SKIPPED!! ***


In [14]:
"""
#  자체 스코어를 파악해 본다 = 정답률 60.37 % 
# - True     4694
# - False    3081
"""

df_score =  submission.copy()             # deep copy
df_score['Y-pred'] = df_score.winner.apply(lambda x: 1 if x > 0.5 else 0)
df_score['Y'] = y_test
df_score['check-sum'] = (df_score['Y-pred'] == df_score['Y'])

_ = df_score['check-sum'].value_counts().to_list()
score = _[0]/(_[0]+_[1])
print(f"*** SCORE = {score * 100:0.7} %")

*** SCORE = 60.25723 %


# LGBM Parametric Study

| NUM | SEED | RANDOM | ITERATION | n_ESTIMATION | num_LEAF | TIME_SPEND| PREDICTION |
|:---:|-----:|---:|---:|-----:|-----:|----------:|:--------:|
| 1   | 50000| 20 | 50 | 1024 | 1024 | 47.28 min | 60.369 % |
| 2   |  4321| 20 | 50 |  500 |  500 | 20.37 min | 60.617 % |
| 3   |  4321| 20 | 50 |  250 |  250 |  8.00 min | 60.758 % |
| 4   |  4321| 20 | 50 |  125 |  125 |  3.29 min | 60.694 % |
| 5   |  4321| 20 | 50 |   62 |   62 |  1.55 min | 60.694 % |
|  -  |
| 1   | 50000| 15 | 75 | 1024 | 1024 | 52.20 min | 60.681 % |
| 2   |  4321| 15 | 75 |  500 |  500 | 24.15 min | 60.553 % |
| 3   |  4321| 15 | 75 |  250 |  250 |  9.48 min | 60.591 % |
| 4   |  4321| 15 | 75 |  125 |  125 |  5.06 min | 60.501 % |
| 5   |  4321| 15 | 75 |   62 |   62 |  2.47 min | 60.360 % |

# 결과 확인하기
- 3월 데이콘 대회 홈페이지 = https://bit.ly/39bqWVg
- 결과 제출하기 = https://dacon.io/competitions/official/235583/mysubmission/

In [15]:
submission

,winner
0,0.451333
1,0.642882
2,0.366945
3,0.365370
4,0.751005
...,...
7770,0.364514
7771,0.365739
7772,0.584126
7773,0.712685


In [16]:
df_score.head(20)

,winner,Y-pred,Y,check-sum
0,0.451333,0,0,True
1,0.642882,1,1,True
2,0.366945,0,0,True
3,0.365370,0,0,True
4,0.751005,1,0,False
5,0.561527,1,0,False
6,0.433676,0,0,True
7,0.541490,1,0,False
8,0.742024,1,1,True
9,0.539299,1,0,False
